# MLP
Test of simple MLP models with different amount of neurons, hidden layers and size of input vector
## Load Data

In [41]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
 
import pandas as pd
import numpy as np

# from tqdm import tqdm

import plotly.express as px
from torch.optim import Adam


import matplotlib.pyplot as plt

In [42]:
df = pd.read_csv("data_N_3000_noise_level_0.05.csv")
df_diff = pd.DataFrame(df.iloc[1:].values - df.iloc[:-1].values, columns=["dt", "dx", "dy", "dz"])

In [43]:
class SequentDataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame, n_dots=1, n_dot_parameters=4):
        self.n_dot_parameters = n_dot_parameters
        self.n_dots = n_dots
        all_data = self.__make_stack(dataframe)
        self.X_ = all_data[:, :-n_dot_parameters]
        self.y_ = all_data[:, -n_dot_parameters:]
        
    def __len__(self):
        return len(self.X_)
    
    def __getitem__(self, idx):
        return self.X_[idx], self.y_[idx]
    
    def __make_stack(self, df: pd.DataFrame):
        stacks = [df.iloc[:-self.n_dots]] + [df.iloc[i:].values if (self.n_dots == i) else df.iloc[i:-(self.n_dots - i)].values for i in range(1, self.n_dots + 1)]
        return torch.tensor(np.hstack(stacks), dtype=torch.float32)
    
    def plotData(self, i = 0, f = -1):
        fig = px.line_3d(x = self.X_[i:f, 1], y = self.X_[i:f, 2], z = self.X_[i:f, 3])
        fig.show()

In [44]:
N_DOTS = 100
N_DOT_PARAMETERS = 4

In [45]:
dataset = SequentDataset(df_diff[0:2500], n_dots=N_DOTS, n_dot_parameters=N_DOT_PARAMETERS)
test = SequentDataset(df_diff[2500:3000], n_dots=N_DOTS, n_dot_parameters=N_DOT_PARAMETERS)
dataset.plotData()

In [46]:
train_data, val_data = random_split(dataset,[0.8,0.2])

train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
val_loader = DataLoader(val_data, batch_size=8, shuffle=False)

## Create Model

In [47]:
class MLPModel(nn.Module):
    def __init__(self, input_dots_amount, input_dot_parameters_amount, output_dots_amount, output_dot_parameters_amount, hidden_layers_lengths, dropout_probability=0, activation=nn.ReLU()):
        super(MLPModel, self).__init__()
        self.layers = nn.ModuleList()
        layer_lengths = [input_dots_amount * input_dot_parameters_amount] + hidden_layers_lengths + [output_dots_amount * output_dot_parameters_amount]
        #self.dropout = nn.Dropout(dropout_probability)
        self.activation = activation
        for i in range(1, len(layer_lengths)):
            self.layers.append(nn.Linear(layer_lengths[i - 1], layer_lengths[i]))
    
    
    def forward(self, X: torch.Tensor):
        X = X.clone()
        for layer in self.layers[:-1]:
            X = layer(X)
            X = self.activation(X)
            #X = self.dropout(X)
        return self.layers[-1](X)

In [48]:
model = MLPModel(input_dots_amount=N_DOTS, input_dot_parameters_amount=N_DOT_PARAMETERS, output_dots_amount=1, output_dot_parameters_amount=4, hidden_layers_lengths=[100, 20, 50])
loss_model = nn.HuberLoss()
opt = Adam(model.parameters(), lr=0.0001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=5, gamma=0.95)

In [49]:
EPOCH = 50
train_losses = []
val_losses = []

for epoch in range(EPOCH):

    # Обучение
    model.train()
    # train_loop = tqdm(train_loader,leave=False)
    train_loss = []
    for X, y in train_loader:
        pred = model(X)
        loss = loss_model(pred, y)
        train_loss.append(loss.item())

        opt.zero_grad()
        loss.backward()

        opt.step()
        mean_train_loss = sum(train_loss)/len(train_loss)
        # train_loop.set_description(f"Epoch [{epoch+1}/{EPOCH}], train_loss = {mean_train_loss:.4f}")

    train_losses.append(mean_train_loss)
    
    # Валидация
    model.eval()
    with torch.no_grad():
        val_loss = []
        for X, y in val_loader:
            pred = model(X)
            loss = loss_model(pred, y)
            val_loss.append(loss.item())

        mean_val_loss = sum(val_loss)/len(val_loss)
        val_losses.append(mean_val_loss)

    lr_scheduler.step()
    lr = lr_scheduler.get_last_lr()
    print(f"Epoch [{epoch+1}/{EPOCH}], train_loss = {mean_train_loss:.4f}, val_loss = {mean_val_loss:.4f}")

Epoch [1/50], train_loss = 0.0751, val_loss = 0.0013
Epoch [2/50], train_loss = 0.0013, val_loss = 0.0013
Epoch [3/50], train_loss = 0.0012, val_loss = 0.0013
Epoch [4/50], train_loss = 0.0012, val_loss = 0.0012
Epoch [5/50], train_loss = 0.0011, val_loss = 0.0012
Epoch [6/50], train_loss = 0.0011, val_loss = 0.0011
Epoch [7/50], train_loss = 0.0011, val_loss = 0.0011
Epoch [8/50], train_loss = 0.0011, val_loss = 0.0011
Epoch [9/50], train_loss = 0.0010, val_loss = 0.0010
Epoch [10/50], train_loss = 0.0010, val_loss = 0.0010
Epoch [11/50], train_loss = 0.0010, val_loss = 0.0010
Epoch [12/50], train_loss = 0.0009, val_loss = 0.0010
Epoch [13/50], train_loss = 0.0009, val_loss = 0.0009
Epoch [14/50], train_loss = 0.0009, val_loss = 0.0009
Epoch [15/50], train_loss = 0.0009, val_loss = 0.0009
Epoch [16/50], train_loss = 0.0009, val_loss = 0.0009
Epoch [17/50], train_loss = 0.0009, val_loss = 0.0009
Epoch [18/50], train_loss = 0.0009, val_loss = 0.0009
Epoch [19/50], train_loss = 0.0008, v

In [50]:
start, _  = test[0]
predictions = []
model.eval()

for i in range(1,500):
    with torch.no_grad():
        pred = model(start)
        predictions.append(pred)
        start = torch.hstack([start[N_DOT_PARAMETERS:], pred]) 
        # start = pred   

In [51]:
p_x = pd.DataFrame(map(lambda t: t.detach().tolist(), predictions), columns=["Time", "X", "Y", "Z"])
_, y  = test[:500]
p_y = pd.DataFrame(y.detach().numpy(), columns=["Time", "X", "Y", "Z"])

p_x = p_x.cumsum()
p_y = p_y.cumsum()

In [52]:
p_x["C"] = "Predict"
p_y["C"] = "True"

d = pd.concat([p_x,p_y],axis=0)

In [53]:
fig = px.line_3d(d, x="X", y="Y", z="Z", color='C')
# fig = px.scatter_3d(data, x="X", y="Y", z="Z")
fig.show()